In [ ]:
import gemmi as gm
import numpy as np
import matplotlib.pyplot as plt

path = "/Users/alejandro/Desktop/mddocs/processing-analysis/PDBs-Codes/fibrilGen/"
filepath = path + "collagen-molecule.pdb"

st = gm.read_structure(filepath)
cell = st.cell
m0 = st[0]                      # base model

# Shift list
additions = [[0,0,0],[1,0,0], [0,1,0], [-1,0,0], [0,-1,0],
             [1,1,0], [-1,1,0], [1,-1,0], [-1,-1,0],

             [0,0,1], [1,0,1], [-1,0,1], [0,1,1], [0,-1,1],
             [1,1,1], [-1,1,1], [1,-1,1], [-1,-1,1],

             [0,0,-1], [1,0,-1], [-1,0,-1], [0,1,-1], [0,-1,-1],
             [1,1,-1], [-1,1,-1], [1,-1,-1], [-1,-1,-1]]
models = []


for addition in additions:
    m_new = 0

    # Get the translation vector - x||a, constructs over this
    shift_frac = gm.Fractional(addition[0], addition[1], addition[2])

    # convert to Cartesian (Å) using the actual cell (handles non-90° cells too)
    shift_cart = cell.orthogonalize(shift_frac)
    ta, tb, tc = shift_cart.tolist()
    tvec = gm.Vec3(ta, tb, tc)

    # build a Transform: identity rotation + translation vector
    T = gm.Transform(gm.Mat33(), tvec)

    # clone and move
    m_new = m0.clone()
    m_new.transform_pos_and_adp(T)       # moves atoms; ADPs handled correctly (rotation-aware)
    models.append(m_new)

# generate model - joining the 2 models
out = gm.Structure()
out.cell = st.cell
out.spacegroup_hm = st.spacegroup_hm

# Create model 0
out.add_model(gm.Model("0"))
mdl = out[0]

# Function to make a single model
def add_model_into_single(adding_model, main_model):
    for ch in adding_model:
        ch2add = ch.clone()
        ch2add.name = ''
        main_model.add_chain(ch2add)

# add_model_into_single(m0, mdl)
for i, model in enumerate(models):
    add_model_into_single(model, mdl)

out.write_pdb(path + "colstruct.pdb")